In [ ]:
import sys
if True:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    root = "/content/drive/My Drive/Colab Notebooks/burgers/"
    sys.path.append("/content/drive/My Drive/Colab Notebooks/burgers/")
else:
    root = './'

In [ ]:
import numpy as np
import torch
from matplotlib import pylab as plt
if torch.cuda.device_count()>0:
    device = torch.device('cuda')
    print("Connected to a GPU")
else:
    print("Using the CPU")
    device = torch.device('cpu')

In [ ]:
import importlib
import util
importlib.reload(util)
from util import *

In [ ]:
def train_it(dataset, model, disc,Npast=1, Nfuture=1):
    #model = model
    learning_rate = 1e-4
    optim_model = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optim_disc  = torch.optim.Adam(disc.parameters(), lr=learning_rate)

    Nepoch = 1000
    Nbatch = 100
    Ntotal = dataset.shape[0]*(dataset.shape[1]-(Npast+1)-Nfuture)
    Niter = Nepoch * Ntotal//Nbatch
    Nprint = (Niter)//10
    Nsave = 100
    losses = np.zeros((Niter//Nsave,2))
    
    for e in range(Niter):
        xx,yy = get_batch(Nbatch, dataset)
        
        # Step the discriminator
        L = - disc(y_pred) + disc(yy[:,:,1:-1])
        optim_disc.zero_grad()
        L.backward()
        optim_disc.step()
        
        # Step the model
        y_pred = model(xx)+xx[:,:,1:-1]
        L = disc(y_pred)
        optim_model.zero_grad()
        L.backward()
        optim_model.step()
        
        yy[:,:,1:-1]
        if e%Nprint == Nprint-1:
            print(e,L.item())
        if e%Nsave == Nsave-1:
            losses[e//Nsave,0] = L.item()
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
    return losses